1 赛题介绍
《绝地求生》(PUBG) 是一款战术竞技型射击类沙盒游戏。在该游戏中，玩家需要在游戏地图上收集各种资源，并在不断缩小的安全区域内对抗其他玩家，让自己生存到最后。当选手在本局游戏中取得第一名后，会有一段台词出现：“大吉大利，晚上吃鸡!”。

在本次赛题中，我们收集了PUBG比赛数据中玩家的行为数据，希望选手能够构建模型对玩家每局最终的排名进行预测。

2 赛事任务
构建吃鸡排名预测模型，输入每位玩家的统计信息、队友统计信息、本局其他玩家的统计信息，预测最终的游戏排名。这里的排名是按照队伍排名，若多位玩家在PUBG一局游戏中组队，则最终排名相同。

赛题训练集案例如下： 训练集5万局数据，共150w行 测试集共5000局条数据，共50w行

赛题数据文件总大小150MB，数据均为csv格式，列使用逗号分割。若使用Pandas读取数据，可参考如下代码：

In [ ]:
import pandas as pd
import numpy as np

pubg_train = pd.read_csv('data/pubg_train.csv')
# 查找缺失值
pubg_train.info()

测试集中label字段team_placement为空，需要选手预测。完整的数据字段含义如下：

match_id：本局游戏的id
team_id：本局游戏中队伍id，表示在每局游戏中队伍信息
game_size：本局队伍数量
party_size：本局游戏中队伍人数
player_assists：玩家助攻数
player_dbno：玩家击倒数
player_dist_ride：玩家车辆行驶距离
player_dist_walk：玩家不幸距离
player_dmg：输出伤害值
player_kills：玩家击杀数
player_name：玩家名称，在训练集和测试集中全局唯一
kill_distance_x_min：击杀另一位选手时最小的x坐标间隔
kill_distance_x_max：击杀另一位选手时最大的x坐标间隔
kill_distance_y_min：击杀另一位选手时最小的y坐标间隔
kill_distance_y_max：击杀另一位选手时最大的x坐标间隔
team_placement：队伍排名
选手需要提交测试集队伍排名预测，具体的提交格式如下：

team_placement
19
19
37
37
49
49
13
13

3 评估指标
本次竞赛的使用绝对回归误差MAE进行评分，数值越低精度越高，评估代码参考：

In [ ]:
from sklearn.metrics import mean_absolute_error
y_pred = [0, 2, 1, 3]
y_true = [0, 1, 2, 3]
100 - mean_absolute_error(y_true, y_pred)

4 数据分析
结合已有的赛题信息，接下来我们将深入分析数据内部的规律，找出什么类型的队伍会取得更好的排名？

赛题字段分析
赛题标签分析
字段相关性分析

In [ ]:
import pandas as pd
import paddle

%pylab inline
import seaborn as sns

train_df = pd.read_csv('data/pubg_train.csv')
test_df = pd.read_csv('data/pubg_test.csv')

In [ ]:

# 查看训练集 验证集
train_df.shape, test_df.shape

In [ ]:
train_df.head(5)

In [ ]:
# 热力图查看数据关联性
sns.heatmap(train_df.corr())

In [ ]:
train_df.columns

    5 模型训练与验证
数据处理

In [ ]:

train_df = train_df.drop(['match_id', 'team_id'], axis=1)
test_df = test_df.drop(['match_id', 'team_id'], axis=1)
# 填充nan值，避免训练出错
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

In [ ]:
# 标签归一化，按照本场比赛的队伍数量进行处理
# train_df['team_placement'] /= train_df['game_size'] 

# 数值归一化 将每一列数据除以最大值
for col in train_df.columns[:-1]:
    train_df[col] /= train_df[col].max()
    test_df[col] /= test_df[col].max()

模型搭建

In [ ]:
class Regressor(paddle.nn.Layer):
    # self代表类的实例自身
    def __init__(self):
        # 初始化父类中的一些参数
        super(Regressor, self).__init__()
        
        # 网络参数（调整此处获得不同模型）
        self.fc1 = paddle.nn.Linear(in_features=13, out_features=32)
        self.fc2 = paddle.nn.Linear(in_features=32, out_features=128)
        self.fc3 = paddle.nn.Linear(in_features=128, out_features=1)
        
        # 在(o,res)之间取最大值作为结果
        self.relu = paddle.nn.ReLU()
    
    # 网络的前向计算
    def forward(self, inputs):
        x = self.fc1(inputs)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        return x

In [ ]:
# 声明定义好的线性回归模型 实例化
model = Regressor()

# 开启模型训练模式
model.train()

# 定义优化算法，使用随机梯度下降SGD
opt = paddle.optimizer.SGD(learning_rate=0.01, parameters=model.parameters())

模型训练

In [ ]:
EPOCH_NUM = 1500   # 设置外层循环次数 尽量大
BATCH_SIZE = 1500  # 设置batch大小  根据硬件调整
# 优化算法
training_data = train_df.iloc[:-10000].values.astype(np.float32)
val_data = train_df.iloc[-10000:].values.astype(np.float32)

# 定义外层循环
for epoch_id in range(EPOCH_NUM):
    # 在每轮迭代开始之前，将训练数据的顺序随机的打乱
    np.random.shuffle(training_data)
    
    # 将训练数据进行拆分，每个batch包含10条数据
    mini_batches = [training_data[k:k+BATCH_SIZE] for k in range(0, len(training_data), BATCH_SIZE)]
    
    train_loss = []
    for iter_id, mini_batch in enumerate(mini_batches):
        # 清空梯度变量，以备下一轮计算
        opt.clear_grad()

        x = np.array(mini_batch[:, :-1])
        y = np.array(mini_batch[:, -1:])
        
        # 将numpy数据转为飞桨动态图tensor的格式
        features = paddle.to_tensor(x)
        y = paddle.to_tensor(y)
        
        # 前向计算
        predicts = model(features)
        
        # 计算损失
        loss = paddle.nn.functional.l1_loss(predicts, label=y)
        avg_loss = paddle.mean(loss)
        train_loss.append(avg_loss.numpy())
        
        # 反向传播，计算每层参数的梯度值
        avg_loss.backward()

        # 更新参数，根据设置好的学习率迭代一步
        opt.step()
    
    mini_batches = [val_data[k:k+BATCH_SIZE] for k in range(0, len(val_data), BATCH_SIZE)]
    val_loss = []
    for iter_id, mini_batch in enumerate(mini_batches):
        x = np.array(mini_batch[:, :-1])
        y = np.array(mini_batch[:, -1:])
        
        features = paddle.to_tensor(x)
        y = paddle.to_tensor(y)
        
        predicts = model(features)
        loss = paddle.nn.functional.l1_loss(predicts, label=y)
        avg_loss = paddle.mean(loss)
        val_loss.append(avg_loss.numpy())

    print(f'Epoch {epoch_id}, train MAE {np.mean(train_loss)}, val MAE {np.mean(val_loss)}')

模型预测

In [ ]:
model.eval()
test_data = paddle.to_tensor(test_df.values.astype(np.float32))
test_predict = model(test_data)
test_predict = test_predict.numpy().flatten()
test_predict = test_predict.round().astype(int)

In [ ]:
pd.DataFrame({'team_placement': test_predict}).to_csv('data/submission.csv', index=None)

# 在左侧刷新下载到桌面后提交进行评测即可

6 总结与展望
项目使用全连接网络进行训练和预测。

后续改进方法有：

按照队伍进行聚合统计数据，构造新特征。
将标签归一化到0-1之间，进行训练。